In [1]:
from plotnine import *
import pandas as pd
import numpy as np
import itables
from IPython.display import Markdown as md
from IPython.display import display, HTML
from datetime import datetime
import logging
import os
import sys

# import etl.env

In [ ]:
HTML(
    """
<style>
    canvas.marks { display: block; margin: auto; }
    div.vega-embed { width: 100%; }
</style>
"""
)

In [ ]:
# plotnine theme variables
# p9_color = "#117da5"
# p9_font = "sans-serif"
# p9_point = 5
# p9_title = 25
# p9_axis_label = 16
# p9_axis_text = 10
# p9_fig_hg = theme(figure_size=(12.8, 14.4))
# p9_fig_lg = theme(figure_size=(12.8, 9.6))
# p9_fig_md = theme(figure_size=(12.8, 4.8))
# p9_fig_sm = theme(figure_size=(12.8, 2.4))
# p9_theme = theme_light(base_size=15, base_family=p9_font) + theme(
#     plot_title=element_text(size=p9_title, face="bold"),
#     axis_title=element_text(size=p9_axis_label, face="italic"),
#     axis_text=element_text(size=p9_axis_text),
# )

In [ ]:
logger = logging.getLogger("root")
logging.basicConfig(
    format="\033[1;36m(def %(funcName)s %(lineno)s): \033[1;37m %(message)s",
    level=logging.INFO,
)

# let's display at least 100 rows by default
pd.set_option("display.max_rows", 1000)

# let's display at least 100 columns by default
pd.set_option("display.max_columns", 1000)

# get today's date
CURRENT_DAY = datetime.today().strftime("%b. %-d, %Y")

# get the current month
CURRENT_MONTH = datetime.now().strftime("%B")

# get the current year
CURRENT_YEAR = datetime.now().year

In [ ]:
def generate_summary(df, grouper, aggregate):
    """
    Given a dataframe and a column
    to group by, outputs topline summary
    stats for each column to group by in a year
    """

    # get a list of the unique values in a dataframe column
    values_list = df[aggregate].unique()

    # create a holding container
    output_data = []

    # loop through our list
    for value in values_list:

        # create a holding dictionary
        output = {}

        # isolate for the value
        iso_df = df.query(f"{aggregate} == '{value}'").reindex()

        # get a count for each month
        grouped = iso_df.groupby(grouper).agg(stat=(aggregate, "count")).reset_index()

        # this is the month
        output["value"] = value

        # find the overall count
        output["count"] = grouped.sum()["stat"]

        # find the mean
        output["mean"] = grouped.mean()["stat"]

        # find the median
        output["median"] = grouped.median()["stat"]

        # find the minimum value
        output["min"] = grouped.min()["stat"]

        # find the year corresponding to the minimum value
        min_year = grouped[grouped["stat"] == output["min"]][grouper].values

        # store the year in the output dictionary
        output["min_year"] = ",".join([str(i) for i in min_year])

        # find the maximum value
        output["max"] = grouped.max()["stat"]

        # find the year corresponding to the minimum value
        max_year = grouped[grouped["stat"] == output["max"]][grouper].values

        # store the year in the output dictionary
        output["max_year"] = ",".join([str(i) for i in max_year])

        # append to our holding container
        output_data.append(output)

    # convert the list to a dataframe
    summary_stats = pd.DataFrame(output_data)

    # return the dataframe
    return summary_stats.sort_values(by="value", ascending=False)

In [ ]:
# build the file path
ANALYSIS_FILE = os.path.join("", "")

# open the file in a dataframe
df = pd.read_csv(
    ANALYSIS_FILE,
    low_memory=False,
    encoding="ISO-8859-1",
)